In [1]:
import configparser, os, re
config = configparser.ConfigParser()
config.read('./keys.ini')
os.environ['GOOGLE_API_KEY'] = config['GOOGLE']['GOOGLE_API_KEY']
os.environ['GOOGLE_CSE_ID'] = config['GOOGLE']['GOOGLE_CSE_ID']
openai_api_key = config['OPENAI']['OPENAI_API_KEY']
os.environ['OPENAI_API_KEY'] = openai_api_key

In [2]:
from typing import Any, List, Mapping, Optional
from langchain.llms.base import LLM
from langchain.callbacks.manager import CallbackManagerForLLMRun
import requests

class CustomLLM2(LLM):
    @property
    def _llm_type(self) -> str:
        return "custom"

    def _call(self, prompt: str, stop: Optional[List[str]] = None) -> str:
        if isinstance(stop, list):
            stop = stop + ["\n###","\nObservation:","\n问题","\nQuestion:"]
        HOST = 'localhost:5000'
        URI = f'http://{HOST}/v1/chat/completions'

        response = requests.post(
            URI,
            json={
                "messages": [
                {
                    "role": "user",
                    "content": prompt
                  }
                ],
                "mode": "instruct",
                "instruction_template": "Alpaca",
            },
        )
        response.raise_for_status()
        return response.json()['choices'][0]['message']['content']
  
    @property
    def _identifying_params(self) -> Mapping[str, Any]:
        """Get the identifying parameters."""
        return {}

#use one 1 chain to summary the story dialogue from main page content of the URL
def story_summary_stuff(docs, prompt_template = ""):
    #input: docs of the web page
    from langchain.chains.llm import LLMChain
    from langchain.prompts import PromptTemplate
    from langchain.chains.combine_documents.stuff import StuffDocumentsChain
    # Define prompt
    if prompt_template == "":
        prompt_template = """The text below is the subtitle of a video about US stock market. Summarize it. 
        List the major sections of the video with time stamp.
        Example output:
        Overall summary of the video
        Time Stamp1: Summary1
        Time Stamp2: Summary2
        ...
        "{subtitle}"
        Output:"""
    prompt = PromptTemplate.from_template(prompt_template)

    # Define LLM chain
    #llm = ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo-16k")
    llm_chain = LLMChain(llm=llm, prompt=prompt)

    # Define StuffDocumentsChain
    stuff_chain = StuffDocumentsChain(
        llm_chain=llm_chain, document_variable_name="subtitle"
    )
    
    return stuff_chain.run(docs)

#youtube loader and return docs for the transcript with time stamps
def loader(link:str, language=["zh"], db_loc = "./cache/YTDBT", overwrite = False):
    #load video info from link
    from langchain.document_loaders import YoutubeLoader
    try:
        loader = YoutubeLoader.from_youtube_url(
            link, add_video_info=True, language=language
        )
        docs = loader.load()
    except Exception as e:
        print(e)
        return(None);
        
    #check whether the vid is already in the DB
    this_id = docs[0].metadata['source']
    from langchain.vectorstores import Chroma
    from langchain.embeddings import OpenAIEmbeddings
    db = Chroma(persist_directory=db_loc, embedding_function=OpenAIEmbeddings())
    tmp = db.get()['ids']
    #this_db_list = [x.split("_")[0] for x in tmp]
    #this_db_set = set(this_db_list)
    this_db_set = set(tmp)
    if this_id in this_db_set:
        if overwrite == False:
            return(link+" link already in the db, skip");
    print("adding link "+link);
            
    #format the transcript into SRT
    from youtube_transcript_api import YouTubeTranscriptApi
    from youtube_transcript_api.formatters import SRTFormatter
    transcript = YouTubeTranscriptApi.get_transcript(docs[0].metadata['source'],languages=language)
    formatter = SRTFormatter()
    srt_formatted = formatter.format_transcript(transcript)
    with open("./cache/Output.srt", "w", encoding="utf-8") as text_file:
        text_file.write(srt_formatted)
    
    #summarize the SRT with time stamps
    from langchain.chat_models import ChatOpenAI
    #llm = ChatOpenAI(temperature=0, model_name="gpt-4-1106-preview")
    from langchain.docstore.document import Document
    output_srt = Document(page_content="Title: "+docs[0].metadata['title']+" \nSRT: "+srt_formatted, metadata=docs[0].metadata);
    res = story_summary_stuff([output_srt])
    
    #write the summary into DB
    from langchain.embeddings import OpenAIEmbeddings
    from langchain.docstore.document import Document
    output_doc = Document(page_content=res, metadata=docs[0].metadata);
    db.add_documents([output_doc], ids = [docs[0].metadata['source']])
    return(output_doc)


In [4]:
from langchain.chat_models import ChatOpenAI
#llm = ChatOpenAI(temperature=0, model_name="gpt-4-1106-preview")
llm = CustomLLM2()
res = loader("https://www.youtube.com/watch?v=esLAnUySOxY&ab_channel=BWB-BusinessWithBrian",language=["en","zh-Hant"], db_loc = "./cache/test", overwrite = True)
print(res.page_content)

adding link https://www.youtube.com/watch?v=esLAnUySOxY&ab_channel=BWB-BusinessWithBrian
The requested instruction is to write a response to the requirements set forth. The response is: Hello, this is Jane Doe. I am calling to express my interest in writing a blog post for your company. I have experience in writing and publishing blog posts that have a wide reach and gain popularity. In addition to my experience in writing, I have also worked with various audiences in the marketing and sales industries for the past ten years. I am confident that I can provide valuable insights that can help your company. If you are interested in having me write for your blog, then please reach out to me. I look forward to hearing from you.


In [15]:
from pytube import Playlist
p = Playlist('https://www.youtube.com/watch?v=whqlwGpMGsA&list=PLn2_heBLYy7-92SSW4nCc3i71-J85Dp5T&ab_channel=TeyvatHistoria')
for url in p.video_urls:
    res = loader(url,["en"])
    print(res.metadata['title'])

Genshin Impact lore - Fischl - Who is the Prinzessin Der Verurteilung?
Genshin Impact - The Origins of Diluc
Genshin Impact lore - The Story of Storm Terror


NoTranscriptFound: 
Could not retrieve a transcript for the video https://www.youtube.com/watch?v=VdoqtjQJcZ0! This is most likely caused by:

No transcripts were found for any of the requested language codes: ['en']

For this video (VdoqtjQJcZ0) transcripts are available in the following languages:

(MANUALLY CREATED)
None

(GENERATED)
 - vi ("Vietnamese (auto-generated)")[TRANSLATABLE]

(TRANSLATION LANGUAGES)
 - af ("Afrikaans")
 - ak ("Akan")
 - sq ("Albanian")
 - am ("Amharic")
 - ar ("Arabic")
 - hy ("Armenian")
 - as ("Assamese")
 - ay ("Aymara")
 - az ("Azerbaijani")
 - bn ("Bangla")
 - eu ("Basque")
 - be ("Belarusian")
 - bho ("Bhojpuri")
 - bs ("Bosnian")
 - bg ("Bulgarian")
 - my ("Burmese")
 - ca ("Catalan")
 - ceb ("Cebuano")
 - zh-Hans ("Chinese (Simplified)")
 - zh-Hant ("Chinese (Traditional)")
 - co ("Corsican")
 - hr ("Croatian")
 - cs ("Czech")
 - da ("Danish")
 - dv ("Divehi")
 - nl ("Dutch")
 - en ("English")
 - eo ("Esperanto")
 - et ("Estonian")
 - ee ("Ewe")
 -

In [5]:
llm = CustomLLM2()
db_loc = "./cache/stock"
Channels = ["https://www.youtube.com/watch?v=0p4EGTMBGoM&list=UULFH-_Z7YSk4QTFmXbYSSBB4w&ab_channel=MikeSwartz",
            "https://www.youtube.com/watch?v=PY2dh6RdfIU&list=UULFoMzWLaPjDJBbipihD694pQ&ab_channel=StockMoe",
            "https://www.youtube.com/watch?v=hE-k_SUKGCs&list=UULFFCEuCsyWP0YkP3CZ3Mr01Q&ab_channel=ThePlainBagel",
            "https://www.youtube.com/watch?v=t1Ic3f8q-o4&list=UULFFhJ8ZFg9W4kLwFTBBNIjOw&ab_channel=NaNa%E8%AF%B4%E7%BE%8E%E8%82%A1",
            "https://www.youtube.com/watch?v=tLLzdsLTDPs&list=UULF7kCeZ53sli_9XwuQeFxLqw&ab_channel=TickerSymbol%3AYOU",
            "https://www.youtube.com/watch?v=7BnzPu6LbMY&list=UULFyqlbzLoYtpqDXwRI9Yh5LA&ab_channel=BWB-BusinessWithBrian",
           ]

from pytube import Playlist
text = ""
for c in Channels: 
    p = Playlist(c)
    url = p.video_urls[0]
    res = loader(url,language=["en","zh-Hant"],db_loc = "./cache/stock")
    if (type(res)==str) | (res == None):
        print(res)
    else:
        print(res.metadata['title'])
        print(res.page_content)
        text = text +"\n" + res.page_content


adding link https://www.youtube.com/watch?v=JdLsSknGQHQ
Wall Street Top Bears Says....

Overall summary of the video:
The video discusses the current state of the stock market and highlights a news article where a top bear on Big Lots sees the stock potentially dropping to $1 single dollar. The video mentions that Big Lots was once a $60 stock but is now trading at $3.82. It also talks about the fear and greed index, which is sitting at 78, indicating no fear in the market. The video then discusses the potential movement in the markets due to upcoming earnings and data releases for several stocks. The dollar index, which was slightly up, and gold were both filling gaps to the downside. The markets were expected to be important due to several earnings releases and economic data. There was also a 84.5% chance the FED would do nothing at their next meeting, and there were growing expectations of rate cuts for this year. Finally, the video discusses trend lines, and the potential for a cor

KeyboardInterrupt: 

In [ ]:
from voiceEN import *

In [15]:
text = text.replace('\r', ' ').replace('\n', ' ')
tts.tts_to_file(text=text, file_path="./cache/stock.wav")

'./cache/stock.wav'